# Initialize libraries

In [2]:
import deeplabcut
from datetime import datetime

from dlchelperclass import DlcHelperClass as dhc

# Create new project

In [ ]:
# Enter your project name
project_name = 'one_vid_res50'
# If you want to use labeled frames from an earlier project,
# make sure to use the same experiment name as in the previous project.
# Enter your experimenter name
experimenter_name = 'liu'
# Specify where you want your project folder to be created
work_dir = '/local/data2/LIA_LIU_PONTUS/LIA_LIU'
# create project path
project_path = work_dir + '/' + project_name + '-' + experimenter_name + '-' + datetime.now().strftime("%Y-%m-%d")
# video path
videos_dir_path = '/local/data2/Cropped_video'
# Creates a list variable of all videos paths 
video_path = dhc.get_video_paths(path=videos_dir_path)
# Create project
deeplabcut.create_new_project(
    project=project_name,
    experimenter=experimenter_name,
    videos=video_path,
    working_directory=work_dir,
    copy_videos=True
)

# Config yaml path
config_path = project_path + '/config.yaml'

# Initialize paths (For loading existing project)
# skip this cell if you created a new project

In [5]:
# Enter your project path
project_path = "/local/data2/LIA_LIU_PONTUS/LIA_LIU/one_vid_stride32-liu-2024-11-25"
# Creates path for config file and videos
config_path, video_path = dhc.get_config_and_video_paths(project_path=project_path)

# Extract frames

In [ ]:
# Extract frames for labeling, amount of images and train/test ratio defined in config.yaml

# "Do you want to extract (perhaps additional) frames for video:" answer yes

deeplabcut.extract_frames(config=config_path, mode='automatic')

# Use napari for labeling 

# Creates CSV and h5 files needed for labeling

In [6]:
deeplabcut.create_training_dataset(config=config_path)

The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.9,
  1,
  (array([14, 12,  7,  4, 18, 11, 22,  6, 23, 10, 13,  1, 24,  3,  2,  0, 17,
          15,  5, 21, 16, 19]),
   array([20,  9,  8])))]

# Train network

In [ ]:
deeplabcut.train_network(config_path, maxiters=1_000_000)

# Creates training_stats.csv

In [ ]:
# Needed for plotting losses
deeplabcut.evaluate_network(config_path, plotting=True)

# Makes prediction on videos

In [ ]:
deeplabcut.analyze_videos(config_path, videos=video_path)

# Create new videos with the models predicted labels

In [ ]:
deeplabcut.create_labeled_video(config_path, videos=video_path)

# Evaluate models

In [ ]:
# Creates likelihood.txt file and loss/vall_loss plot
dhc.save_mean_likelihood_to_file(project_path=project_path)
dhc.plot_loss_to_png(project_path)

# Extract outlier frames and re-labeling

In [ ]:
deeplabcut.extract_outlier_frames(config=config_path,
                                   shuffle=1,
                                  automatic=True,
                                  videos=video_path)

In [ ]:
# Open the DeepLabCut GUI to manually correct labels
# Launch the GUI to manually adjust labels on the extracted frames
deeplabcut.refine_labels(project_path)

In [ ]:
deeplabcut.merge_datasets(config=config_path)

In [ ]:
# Create a new training dataset that includes the updated labels
# This prepares the training files with the corrected labels for training
deeplabcut.create_training_dataset(config=config_path)

# Finetune 

In [ ]:
# Make sure you have run the following code line before:
# * deeplabcut.analyze_videos(project_path, video_paths)

In [ ]:
# Retrain the model with the updated dataset
# Use transfer learning to improve the existing model with the new data
deeplabcut.train_network(project_path, shuffle=1, displayiters=100, saveiters=1000)

In [ ]:
# Re-evaluate performance on the updated model
# Analyze the videos again to check if model accuracy has improved after retraining
deeplabcut.analyze_videos(project_path, video_path)

# Predict unseen data

In [ ]:
video_path = "/local/data2/LIA_LIU_PONTUS/LIA_LIU/untrained_videos/CROP_stride32_lr0001_bs_1/GH010336_converted_2.mp4"

In [ ]:
deeplabcut.analyze_videos(config_path, videos=video_path)

In [ ]:
deeplabcut.create_labeled_video(config_path, videos=video_path)

In [ ]:
import os

# Replace with your actual old projects directory
old_project_path = "/local/data2/old_projects/"

# Iterate through each project in the old_project_path
for project in os.listdir(old_project_path):
    project_path = os.path.join(old_project_path, project)  # Construct full project path

    # Check if the path is a directory (to ensure it's a project folder)
    if os.path.isdir(project_path):
        # Call methods from your helper class
        dhc.save_mean_likelihood_to_file(project_path=project_path)
        dhc.plot_loss_to_png(project_path)
                

# Get likelihood from predicted video

In [ ]:
import pandas as pd

h5_path = "/local/data2/LIA_LIU_PONTUS/LIA_LIU/untrained_videos/CROP_stride32_lr0001_bs_1/GH010336_converted_2DLC_DlcrnetStride32Ms5_dlcrnet_stride32_ms5_lr_0001_bs_1Nov21shuffle1_snapshot_100.h5"
df = pd.read_hdf(h5_path)
df.columns = [f"{bodypart}_{coord}" for bodypart, coord in zip(df.columns.get_level_values(1), df.columns.get_level_values(2))]
df_monofil = df.loc[:, df.columns.str.startswith(('FR', 'FG', 'FB')) & ~df.columns.str.endswith('likelihood')]
df_square = df.loc[:, df.columns.str.startswith(('Top_left', 'Top_right', 'Bottom_left', 'Bottom_right')) & ~df.columns.str.endswith('likelihood')]
df_likelihoods = df.loc[:, df.columns.str.endswith('likelihood')]
overall_average = df_likelihoods.mean().mean()
print("Overall Average Likelihood:", overall_average)
bodypart_means = df_likelihoods.mean(axis=0)
print("Mean likelihood for each body part:")
print(bodypart_means)